In [103]:
import json
import re
import sys
import pandas as pd
import numpy as np


In [135]:
df_train = pd.read_csv('../storage/train_data/train-250315-01.csv')
df_train.head()

,공고번호,주택명,공급지역코드,공급지역명,공급위치우편번호,공급위치,공급규모,모집공고일,청약접수시작일,청약접수종료일,...,공급금액(최고가 기준),위도,경도,행정동코드,법정동코드,시도,시군구,읍면동1,읍면동2,전용면적당 공급금액(최고가기준)
0,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06
1,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06
2,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06
3,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06
4,2025000043,부천 JY 포에시아,410,경기,14636,경기도 부천시 원미구 신흥로 20,55,2025-02-28,2025-03-10,2025-03-11,...,377000000.0,37.485377,126.776922,4.119252e+09,4.119210e+09,경기,부천시 원미구,심곡2동,심곡동,8.767442e+06


In [136]:
df_train['모집공고일_년월'] = pd.to_datetime(df_train['모집공고일']).dt.strftime('%Y%m').astype(int)
df_train['모집공고일_년월']

0        202502
1        202502
2        202502
3        202502
4        202502
          ...  
19819    202003
19820    202003
19821    202003
19822    202003
19823    202003
Name: 모집공고일_년월, Length: 19824, dtype: int64

In [137]:
df_real_estate_price = pd.read_csv('서울_전체_월별_법정동별_실거래가_평균.csv', encoding='cp949')
df_real_estate_price

,년월,구,법정동,법정동시군구코드,법정동읍면동코드,거래금액(만원),전용면적당 거래금액(만원),법정동코드
0,202501,강남구,개포동,11680,10300,237042.451613,3379.567200,1168010300
1,202501,강남구,논현동,11680,10800,159525.000000,1940.551981,1168010800
2,202501,강남구,대치동,11680,10600,357196.153846,3286.645631,1168010600
3,202501,강남구,도곡동,11680,11800,261500.000000,2646.709886,1168011800
4,202501,강남구,삼성동,11680,10500,266062.500000,2930.899752,1168010500
...,...,...,...,...,...,...,...,...
14747,202012,중랑구,면목동,11260,10100,58877.777778,865.857283,1126010100
14748,202012,중랑구,묵동,11260,10400,69147.222222,887.297813,1126010400
14749,202012,중랑구,상봉동,11260,10200,63295.454545,900.506401,1126010200
14750,202012,중랑구,신내동,11260,10600,57373.943662,842.389505,1126010600


In [ ]:
def apply_price_diff(row):
    b_code = row['법정동코드']
    date = row['모집공고일_년월']
    offer_price = row['전용면적당 공급금액(최고가기준)']

    mask = (df_real_estate_price['법정동코드'] == b_code) & (df_real_estate_price['년월'] == date)
    matched_rows = df_real_estate_price[mask]

    if matched_rows.empty:
        # 매칭된 데이터가 없을 때 기본값 처리 (예: NaN)
        return np.nan

    real_price = matched_rows.iloc[0]['전용면적당 거래금액(만원)']
    price_diff = offer_price - real_price

    return price_diff

df_train['전용면적당 시세차익'] = df_train.apply(apply_price_diff, axis=1)

4119210200.0 0        1168010300
1        1168010800
2        1168010600
3        1168011800
4        1168010500
            ...    
14747    1126010100
14748    1126010400
14749    1126010200
14750    1126010600
14751    1126010300
Name: 법정동코드, Length: 14752, dtype: int64
4119210200.0 0        1168010300
1        1168010800
2        1168010600
3        1168011800
4        1168010500
            ...    
14747    1126010100
14748    1126010400
14749    1126010200
14750    1126010600
14751    1126010300
Name: 법정동코드, Length: 14752, dtype: int64
4119210200.0 0        1168010300
1        1168010800
2        1168010600
3        1168011800
4        1168010500
            ...    
14747    1126010100
14748    1126010400
14749    1126010200
14750    1126010600
14751    1126010300
Name: 법정동코드, Length: 14752, dtype: int64
4119210200.0 0        1168010300
1        1168010800
2        1168010600
3        1168011800
4        1168010500
            ...    
14747    1126010100
14748    1126010400
14749

In [ ]:
df_train[df_train['공급지역명'] == '서울']